### SRA data transfer to GCP Storage



In [ ]:
%run ../config/init.py

### Loading data from {{ cookiecutter.dataset_name }}/sample_table.csv accession list

The file `{{ cookiecutter.dataset_name }}/sample_table.cs` should contains a single column with all SRA IDs to be processed.

In [ ]:
os.chdir(os.path.join(DATA, DATASET))
PIPELINE_JSON = os.path.join(BIN, 'gcp', 'pipeline-download-sra.json')
sra_df = pandas.read_csv(os.path.join(DATA, DATASET, 'sample_table.csv'), header=None)
sra_df

## Testing gcloud configuration

### Requirements

#### [Cloud SDK](https://cloud.google.com/sdk)


Run *gcloud init* to initialize the gcloud environment and follow its instructions:

 `$ gcloud init`

In [ ]:
account = !gcloud config get-value account
account = ''.join(account)
project = !gcloud config get-value project
project = ''.join(project)
if account != '(unset)' and project != '(unset)':
    print('Using account: {} with project: {}'.format(account, project))
else:
    print('Please, configure Cloud SDK before running this notebook')
    print('Open a Terminal and run: gcloud init')

### Defining variables

Edit GCP zone and region variable accordingly to your geographical location.

In [ ]:
REGION = 'us-east4'
ZONE = 'us-east4-c'

### Create or retrieve GCP storage bucket

In [ ]:
bucket_list = !gsutil ls
bucket = None

prefix = 'gs://{}-sra-'.format(DATASET.lower())
for l in bucket_list:
    if prefix in l:
        bucket = l.replace('gs://{}-sra-'.format(DATASET.lower()),'').replace('/','')
        break

if not bucket:
    bucket = str(uuid.uuid4())
    bucket_name = '{}-sra-'.format(DATASET.lower()) + bucket
    !/usr/bin/gsutil mb gs://{bucket_name}

print('bucket: {}-sra-{}'.format(DATASET.lower(), bucket))

### Submitting jobs for transfering the SRA files to GCP

In [ ]:
operations = {}
PIPELINE = os.path.join(BIN, 'gcp', 'pipeline-download-sra.json')

op_dir = os.path.join(DATA, DATASET, 'gcp')
if not os.path.exists(op_dir):
    os.mkdir(op_dir) 
os.chdir(op_dir)
   
bucket_name = '{}-sra-'.format(DATASET.lower()) + bucket
        
if os.path.exists('operations-sra-download.tsv'):
    operations['logs'] = {}
    operations['operations'] = pandas.read_csv('operations-sra-download.tsv', sep='\t')
else:
    d = []
    for f in sra_df[0].unique():
        a = !gcloud beta lifesciences pipelines run --pipeline-file={PIPELINE} --env-vars=OUTBUCKET={bucket_name},SAMPLE={f}
        if len(a) == 1 and a[0].startswith('Running'):
            a = a[0].replace('].','').split('/')[5]
            d.append([f, a, 'running'])
        else:
            d.append([f, None, a])
    operations['logs'] = {}
    operations['operations'] = pandas.DataFrame(d, columns=['sample', 'id', 'status'])
    operations['operations'].to_csv('operations-sra-download.tsv', sep='\t', index=None)

display(operations['operations'])

### GCP log retrival for plotting
This cell will download the GCP logs for completed operations (jobs) creating the `[sample.json` files for each sample. 

You should executed it multiple times until all operations are completed. 

In [ ]:
os.chdir(op_dir)
    
df = operations['operations'].dropna()
data = []                
for i, r in df.iterrows():
    id = r['id']
    if os.path.exists('{}.json.gz'.format(r['sample'])):
        with gzip.GzipFile('{}.json.gz'.format(r['sample']), 'r') as fin:  
            operations['logs'][r['sample']] = json.loads(fin.read().decode('utf-8'))
    else:
        if r['sample'] not in operations['logs']:
            a = !gcloud beta lifesciences operations describe --format=json {id}
            l = json.loads(''.join(a))
            if 'done' in l:
                operations['logs'][r['sample']] = l
                with gzip.GzipFile('{}.json.gz'.format(r['sample']), 'w') as fout:   # 4. gzip
                    fout.write(json.dumps(l, indent=2).encode('utf-8'))  
    if r['sample'] in operations['logs']:
        ts = get_gpc_starttimestamp(operations['logs'][r['sample']])
        ts = datetime.strptime(ts.split('.')[0], "%Y-%m-%dT%H:%M:%S")
        te = datetime.strptime(operations['logs'][r['sample']]['metadata']['endTime'].split('.')[0], "%Y-%m-%dT%H:%M:%S")
        elapsed = te - ts
        data.append([r['sample'], elapsed])
operations['gcp'] = pandas.DataFrame(data, columns=['Sample', 'Time'])
operations['gcp']['Time'] = operations['gcp']['Time']/pandas.Timedelta('1 minute')
display(operations['gcp'])


MACHINE_PRICE = 0.16 # n1-standard-16 preemptible
print('Computing cost cost: $ {:.2f}'.format(operations['gcp']['Time'].sum() * MACHINE_PRICE/60))

## Downloading results from GCP

In [ ]:
os.chdir(os.path.join(DATA, DATASET))
!gsutil -m -o 'GSUtil:parallel_composite_upload_threshold=150M' -o 'GSUtil:parallel_process_count=4' -o 'GSUtil:parallel_thread_count=4' rsync -x '.*\.fastq\.gz' gs://{bucket_name}/ ./ 


In [ ]:
str_msg = '#### FastQC report\n'
display(Markdown(str_msg))
os.chdir(NOTEBOOKS)
data_dir = os.path.join(DATA, DATASET)
base_url = 'https://storage.cloud.google.com/{}-sra-{}/'.format(DATASET.lower(), bucket)

str_msg = '| Sample | FastQC<br>Report | No of Reads<br>in fastq | Seq<br> Len | %GC '
str_msg += '| Poor<br>Quality | Fail<br>Tests |\n'
str_msg += '| --- | --- |--- | --- | --- | --- | --- |\n'
for sample in sra_df[0].unique():
{% if cookiecutter.sequencing_technology == 'paired-end' %}
    for r in range(1,3):
        s = '{}_{}'.format(sample, r)
        str_msg += '| <a href="{0}{1}.fastq.gz" target="_blank">{1}</a>'.format(base_url, s)
        str_msg += '| '
        str_msg += find_file_print_link_size(data_dir, s, '.html', 'MB', ' --- ')
        str_msg += ' |'
        f = os.path.relpath(os.path.join(data_dir, s + '_fastqc.zip'))
        if os.path.exists(f) and os.path.getsize(f) != 0:
            tests, tot_seq, poor_quality, seq_len, gc_content = parse_fastqc_zip(f)            
            str_msg += "{:,}".format(tot_seq) + '|'
            str_msg += seq_len + '|'
            str_msg += gc_content + '|'
            str_msg += str(poor_quality) + '|'
            fail_tests = ''
            for t in tests:
                if tests[t] == 'FAIL':
                    if fail_tests:
                        fail_tests += '<br>'
                    fail_tests += t
            str_msg += fail_tests + '|\n'

        else:
            str_msg += ' --- | --- | --- | --- | --- |\n'
{% else %}
    str_msg += '| <a href="{0}{1}.fastq.gz" target="_blank">{1}</a>'.format(base_url, sample)
    str_msg += '| '
    str_msg += find_file_print_link_size(data_dir, sample, '.html', 'MB', ' --- ')
    str_msg += ' |'
    f = os.path.relpath(os.path.join(data_dir, s + '_fastqc.zip'))
    if os.path.exists(f) and os.path.getsize(f) != 0:
        tests, tot_seq, poor_quality, seq_len, gc_content = parse_fastqc_zip(f)
        str_msg += "{:,}".format(tot_seq) + '|'
        str_msg += seq_len + '|'
        str_msg += gc_content + '|'
        str_msg += str(poor_quality) + '|'
        fail_tests = ''
        for t in tests:
            if tests[t] == 'FAIL':
                if fail_tests:
                    fail_tests += '<br>'
                fail_tests += t
        str_msg += fail_tests + '|\n'

    else:
        str_msg += ' --- | --- | --- | --- | --- |\n'
{% endif %}

display(Markdown(str_msg))
del str_msg